In [ ]:
#-1: Setting up google colab:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
%cd '/content/gdrive/My Drive/CS6910_DL/a4/'
!ls
#!unzip data.zip

In [ ]:
#0. Importing modules & defining global variables:
import os
import csv
import pickle
import random
import string
import numpy as np 
import pandas as pd
import seaborn as sns
import tensorflow as tf 
from sklearn import metrics
from tensorflow import keras
from matplotlib import pyplot as plot 
from tensorflow.keras import backend as bk
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
print(tf.__version__)

dir='./data/data1/'
out_vgg16=4096
K,D=4,3
val_split,max_patience,batchsize,max_epochs=0.1,5,2,100

In [ ]:
#1. Image pre-processing functions:
def req_images(images='Test_image.txt'):  #'image_names.txt'):
    with open(dir+images) as f:
        reader=csv.reader(f,delimiter="\t")
        image_filename=list(reader)
    image_filenames=np.concatenate(image_filename)
    return image_filenames

def image_featurizer(sub_dir='Images',out_pkl='features1.pkl',j_limit=4):  #features.pkil==(4096), features2.pkl==(7,7,512) of vgg16; j_limit=10k.
    features,j=dict(),0
    model1=keras.applications.VGG16()
    model1._layers.pop()
    model1._layers.pop()
    model=keras.models.Model(inputs=model1.inputs, outputs=model1.layers[-3].output)
    model.summary()
    for filename in os.listdir(dir+sub_dir+'/'):
        j=j+1
        if j>j_limit:
            break
        print('Reading ',j,'.',filename,'...')
        img=img_to_array(load_img(dir+'Images/'+filename,target_size=(224,224)))
        img=img.reshape((1,img.shape[0],img.shape[1],img.shape[2]))
        img=keras.applications.vgg16.preprocess_input(img)
        features[filename]=model.predict(img,verbose=0)
    print('Extracted Features: ',len(features))
#    pickle.dump(features, open(out_pkl, 'wb'))

def load_photo_features(dataset,filename='features2.pkl'):
    all_features,features,j = pickle.load(open(filename, 'rb')),dict(),0
    for key, value in all_features.items():
        if key in dataset:
            j=j+1
            print('Reading features of ',j,'.',key,'...')
            features[key]=value
#	features = {k: all_features[k] for k in dataset}
    return features

In [ ]:
#2. Text pre-processing functions:
def to_vocabulary(descriptions):
	vocab = set()
	for key in descriptions.keys():
		[vocab.update(d.split()) for d in descriptions[key]]
	return vocab

def text_extractor(image_filenames,captions='captions.txt'):
	mapping,table,j = dict(),str.maketrans('', '', string.punctuation),0
	print('Total of ',len(image_filenames),' unique images to be read!')
	with open(dir+captions) as f:
		reader=csv.reader(f,delimiter="\t")
		captions_data=list(reader)
	for i in range(len(captions_data)):
		image_filename=str.split(str.split(captions_data[i][0],'#')[0],'.1')[0]
		if image_filename in image_filenames:
			j=j+1
			print('Reading captions of ',j,'.',image_filename,'...')
			if image_filename not in mapping.keys():
				mapping[image_filename]=[]
			description=captions_data[i][1].split()
			description=[word.lower() for word in description]
			description=[word.translate(table) for word in description]
			description=[word for word in description if len(word)>1 and word.isalpha()]
			mapping[image_filename].append('startseq ' + ' '.join(description) + ' endseq')
	print('Length of vocabulary: ',len(to_vocabulary(mapping)))
	return mapping

def all_desc(descriptions):
	all_descriptions = list()
	for key in descriptions.keys():
		[all_descriptions.append(d) for d in descriptions[key]]
	return all_descriptions

def max_len(descriptions):
	lines = all_desc(descriptions)
	return max(len(d.split()) for d in lines)
 
def create_tokenizer(descriptions):
	lines = all_desc(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

def data_generator1(descriptions, photos, tokenizer, max_length, vocab_size):
	# loop for ever over images
	while 1:
		for key, desc_list in descriptions.items():
			# retrieve the photo feature
			photo = photos[key][0]
			in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
			yield [[in_img, in_seq], out_word]
def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
	while 1:
		for key, desc in descriptions.items():
			if key in photos.keys():
				photo = photos[key][0]
				in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc, photo, vocab_size)
				yield [[in_img, in_seq], out_word]

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, descriptions, tokenizer, max_length, photos, vocab_size, batch_size=2, shuffle=True):
        self.descriptions=descriptions
        self.batch_size = batch_size
        self.tokenizer=tokenizer
        self.max_length=max_length
        self.photos=photos
        self.vocab_size=vocab_size
        self.list_IDs = list(self.descriptions.keys())
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        desc=dict()
        for i, ID in enumerate(list_IDs_temp):
            desc[ID]=self.descriptions[ID]
        return create_sequences(self.tokenizer,self.max_length,desc,self.photos,self.vocab_size)


In [ ]:
#3. Classes & functions for building model:
class NetVLADLayer(tf.keras.layers.Layer):
	def __init__(self,num_clusters,**kwargs):
		self.K = num_clusters
		super(NetVLADLayer,self).__init__(**kwargs)

	def build(self,input_shape):
		self.D = input_shape[-1]
		self.kernel = self.add_weight(shape=(1,1,self.D,self.K),initializer='GlorotUniform',trainable=True,name='alpha')
		self.bias = self.add_weight(shape=(1,1,self.K),initializer='uniform',trainable=True,name='bias')
		self.C = self.add_weight(shape=[1,1,1,self.D,self.K],initializer='GlorotUniform',trainable=True,name='beta')
		super(NetVLADLayer,self).build(input_shape)  

	def call(self,x):
		input_shape = tf.shape(x)
		a = bk.expand_dims(bk.softmax(bk.conv2d(x,self.kernel,strides=(1,1),padding='same')+self.bias),-2)
		z_k = a * (bk.expand_dims(x,-1)+self.C)
		z = bk.sum(z_k, axis=[1, 2])
		z = bk.permute_dimensions(z, pattern=[0, 2, 1])
		z = bk.l2_normalize(z,axis=-1)
		z = bk.batch_flatten(z)
		z = bk.l2_normalize(z,axis=-1)
		return tf.reshape(z, self.compute_output_shape(input_shape))

	def compute_output_shape(self,input_shape):
		return (input_shape[0],self.K*self.D)

	def get_config(self):
		pass
		config = {'num_clusters': self.K}
		base_config = super(NetVLADLayer, self).get_config()
		return dict(list(base_config.items()) + list(config.items()))

def build_model(vocab_size,max_length,inp_shape,K=0,drop=0.5,d1=256,embed=256,rnn=256,lstm=256,d2=256):
    input1=keras.Input(shape=inp_shape)
    input2=keras.Input(shape=(max_length,))
    netvlad1=NetVLADLayer(K)
    drop1=keras.layers.Dropout(drop)
    dense1=keras.layers.Dense(d1, activation='softmax')
    embed1=keras.layers.Embedding(vocab_size,embed,mask_zero=True)
    rnn1=keras.layers.SimpleRNN(rnn) 
    lstm1=keras.layers.LSTM(lstm)  #,return_sequences=True)
    dense2=keras.layers.Dense(d2,activation='relu')
    dense3=keras.layers.Dense(vocab_size,activation='softmax')

    out_netvlad1=netvlad1(input1)
    out_dense1=dense1(out_netvlad1)
    out_embed1=embed1(input2)
    out_rnn1=rnn1(drop1(out_embed1))  #lstm1(drop1(out_embed1))
    out_merge1=keras.layers.concatenate([out_rnn1,out_dense1])
    out_dense2=dense2(out_merge1)
    output1=dense3(out_dense2)
    model=keras.models.Model(inputs=[input1,input2],outputs=output1)
    model.compile(loss=keras.losses.CategoricalCrossentropy(),optimizer='adam',metrics=['mae','mse','accuracy'])
#	print(model.summary())
#	plot_model(model, to_file='model.png', show_shapes=True)
    return model


In [ ]:
#4. Create complicated sequences:

def create_sequences(tokenizer, max_length, descriptions, photos, vocab_size):
	imgs, txts, outs = list(), list(), list()
	for key,desc_list in descriptions.items():
		for desc in desc_list:
			seq = tokenizer.texts_to_sequences([desc])[0]
			for i in range(1, len(seq)):
				in_seq, out_seq = seq[:i], seq[i]
				in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
				out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
				imgs.append(photos[key])
				txts.append(in_seq)
				outs.append(out_seq)
	return np.array(imgs), np.array(txts), np.array(outs)


In [ ]:
#5. Data-loading function:
def data():
#    image_featurizer()
    image_filenames=req_images()
    map=text_extractor(image_filenames)
    tokenizer=create_tokenizer(map)
    vocab_size=len(tokenizer.word_index)+1
    max_length=max_len(map)
    print('Max_length of a sentence: ',max_length)
    photos=load_photo_features(image_filenames)
    print('Total photo features available: ',len(photos))
    print('Loaded image features and corresponding captions in sequence successfully!')
    return map, tokenizer, vocab_size, max_length, photos
map,tokenizer,vocab_size,max_length,photos=data()


In [ ]:
#6. Integration:
from itertools import islice
def chunks(data, SIZE=10000):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}
'''
if __name__ == '__main__':
#    training_generator=DataGenerator(map, tokenizer, max_length, photos, vocab_size,batch_size=2)
    inp_shape=photos[list(photos.keys())[0]].shape
    model=tf.keras.models.load_model('./models/a4_1/full_model')
#    model=build_model(vocab_size,max_length,inp_shape[1:],K=K)
    earlystop=keras.callbacks.EarlyStopping(monitor='val_loss',patience=max_patience)
#    print(len(map))
    rm=0
    for i in range(2):
        for item in chunks(map,1000):
            rm=rm+1
            if rm >= 3:
                imgs,txts,outs = create_sequences(tokenizer,max_length,item,photos,vocab_size)
                model.fit([np.reshape(imgs,(imgs.shape[0],7,7,512)),txts],outs,epochs=5,validation_split=0.1,callbacks=[earlystop])
                model.save('./models/a4_1/full_model')
#            model.fit([imgs,txts],outs,epochs=2)
    #for i in range(1):
    #    earlystop=keras.callbacks.EarlyStopping(monitor='val_loss',patience=max_patience)
    
#        create_sequences(tokenizer, max_length, desc, photo, vocab_size)
#    generator=data_generator(map,photos,tokenizer,max_length,vocab_size)
#        inputs,outputs=next(generator)
#        print(inputs)
#        print(outputs)
#        print(inputs[1].shape,outputs.shape)
#    x,y,z=create_sequences(tokenizer,max_length,map,photos,vocab_size)
#    model.fit([x,y],z,epochs=max_epochs)
#    history=model.fit_generator(generator,epochs=1)

#    imgs,txts,outs=create_sequences(tokenizer,max_length,map,photos,vocab_size)
#    print(imgs.shape,txts.shape,outs.shape)
    #model=build_model(vocab_size,max_length,out_vgg16,K=K)
    #earlystop=keras.callbacks.EarlyStopping(monitor='val_loss',patience=max_patience)
    #model.fit([imgs,txts],outs,epochs=max_epochs,batch_size=batchsize,validation_split=val_split,callbacks=[earlystop])

In [ ]:
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = np.argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0, 1.0, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0, 0, 1.0, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0, 0, 0, 1.0))) 
#		print('key',key)
#		print('predicted',predicted[-1])
#		print('actual',actual[-1])
	# calculate BLEU score
#	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
#	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
#	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
#	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))) 
from nltk.translate.bleu_score import corpus_bleu
model=tf.keras.models.load_model('./models/a4_1/full_model')
evaluate_model(model,map,photos,tokenizer,max_length)

In [ ]:
#7. Plots of model results:
def plots(history):
    print(history.history.keys())
    plot.plot(history.history['accuracy'])
    plot.plot(history.history['val_accuracy'])
    plot.title('Model_Accuracy vs #Epochs')
    plot.ylabel('Accuracy')
    plot.xlabel('#Epochs')
    plot.legend(['Train', 'Test'], loc='upper right')
    plot.show()
    plot.plot(history.history['loss'])
    plot.plot(history.history['val_loss'])
    plot.title('Model_Loss (CrossEntropy) vs #Epochs')
    plot.ylabel('Loss')
    plot.xlabel('#Epochs')
    plot.legend(['Train', 'Test'], loc='upper right')
    plot.show()
#plots(history)